<a href="https://colab.research.google.com/github/IvyAldama/EstructurasDeDatos/blob/main/UTEL_IrisIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentes inteligentes

## Descripción del problema

Empecemos a implementar un agente para el siguiente caso: Estás al cuidado de un invernadero, pero claramente no sabes nada de plantas. Solo te han encargado cuidar a 3 tipos de una planta llamada Iris. Como en realidad tu pasión es el desarrollo de agentes y no la botánica se te ha ocurrido colocar un poco de inteligencia artificial. Cada tipo de Iris requiere diferentes niveles de agua entonces determinas que con saber el tipo de planta es más que suficiente para resolver la cantidad de agua. El problema ahora requiere de usar algunos sensores para tomar la decisión sobre la cantidad de agua a utilizar en tres niveles. Estás determinado a probar esto en tu laboratorio con 150 muestras y finalmente desplegarlo en el invernadero de escala industrial. Tus sensores pueden [medir el ancho y largo del sépalo](https://es.wikipedia.org/wiki/S%C3%A9palo) así como del [pétalo](https://es.wikipedia.org/wiki/P%C3%A9talo).


## Implementación de la solución

Vamos a importar las bibliotecas necesarias: pandas para análisis de datos y scikit-learn para descargar la base de datos (después veremos que scikit-learn nos provee de muchas más herramientas).  


In [ ]:
import pandas as pd
from sklearn import datasets

Y ahora descarguemos la base de datos:

In [ ]:
iris_sk = datasets.load_iris(as_frame=True) # descarga base de datos
iris = pd.DataFrame(data=iris_sk.data, columns=iris_sk.feature_names) # transforma a pandas

iris['target'] = pd.Series(iris_sk.target) # genera una nueva columna TARGET con los códigos de las plantas correspondientes
target_names = pd.Series(iris_sk.target_names) # genera una nueva SERIE con los nombres de la plantas correspondientes

La función load_iris va al internet por una versión de esta base de datos: https://archive.ics.uci.edu/dataset/53/iris. El repositorio de UC Irvine es probablemente el más visitado junto a Kaggle para obtener datos. Cuenta con distintos tipos de conjuntos de datos listos para ser incorporados en lenguajes como Python. Es tan famoso y usado, que scikit-learn lo incorpora como una función dentro de su API. Existen otras como wine, breast o car evaluation que muchos científicos alrededor del mundo usan para probar sus agentes.
Dado que la base de datos viene en un formato propio de scikit-learn, es necesario transformarla a Pandas. Para esto se genera un DataFrame con los datos descargados y el nombre de las columnas correspondientes.  
Las últimas dos líneas nos permitirán la evaluación de nuestro modelo. Iris[“target”] genera una nueva columna en el DataFrame con los tipos de planta contados de 0 al 2. Target_names solo obtiene los nombres correspondientes al tipo 0,1 y 2.
Recuerda que cada celda debe ejecutarse. Lo puedes hacer con CTRL+Enter o con el botón de PLAY.


Ahora imprime iris

In [ ]:
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


Y listo, ya tenemos nuestra muestra de 150 plantas que en este momento conformarán el ambiente completo.

Dado que vamos empezando a determinar a los agentes inteligentes, podemos tomar dos caminos:

1. Guardar toda esta base de datos como nuestro aprendizaje y esperar que cualquier planta en el invernadero se parezca a algo que ya tenemos guardado.

2. Tratar de generar algún modelo estadístico o algoritmo que pueda tratar de mapear las entradas de sépalo y pétalo.


Claramente los científicos detrás del ChatGPT no han hecho la opción (1). La razón es que no es escalable. Si un día el tipo de plantas crece, necesitaríamos ir a obtener muchas más muestras. Normalmente optaremos por la opción (2), hacer un modelo. Y para esto vamos a hacer una *combinación lineal*.

Aunque el nombre puede espantar no es más que asignarle una especie de porcentaje a cada valor del estímulo y tratar de ver si la sumatoria de la multiplicación puede dar un valor adecuado.

Si en estos momentos tu pregunta es *¿por qué estamos combinando peras con manzanas?* Si queremos sumar el largo del pétalo con el ancho del sépalo, no existe matemática rara nos permite hacer eso. Recuerda que la salida de la ecuación que vamos a plantear no es un cálculo físico, solo es un mapeo. Matemáticamente podemos darnos esta __libertad creativa__. Igualmente puede haber algo difícil de explicar. NO vamos a limitar de ninguna manera la salida de la ecuación más allá de decir que el valor tiene que ser entero. Es decir, que si por alguna razón sale 3, y solo debíamos de tener 0,1 o 2, se contabilizará el error, pero no haremos mucho para forzarlo.

De esta forma la ecuación quedaría de la siguiente manera:
$f(sepaloL, sepaloA,petaloL,petaloA) = A*sepaloL+B*sepaloA+C*petaloL+D*petaloA$



Como ves, solo estamos haciendo un modelo que basado en los coeficientes A,B,C y D generará una salida. Ahora, para nuestra medida de rendimiento. Cada valor deberá compararse con el valor real (0,1,2) de la base de datos. Al valor obtenido por la función vamos a decirle “h”, y al valor real “y”:

$p(h,y) =
\left\{
	\begin{array}{ll}
		1  & \mbox{if } h \not = y \\
		0 & \mbox{if } h = y
	\end{array}
\right.$

 A este marco de trabajo se le llama PEAS: Performance, Environment, Actuators, Sensors. Es decir, para resolver un problema con un agente inteligente debemos determinar la medida de rendimiento, el ambiente, los actuadores y los sensores.


Para implementar la función del agente:

In [ ]:
# definicion del agente, mapeo de sensores a clasificacion
def agent(a,b,c,d,x):
  return a*x[0] + b*x[1]  + c*x[2] + d*x[3]

x1 = iris.iloc[0] # obtencion del primer registro
print(x1) # imprime el primer registro
h = agent(0,0,0,0,x1) # evalua con datos "sensados"
print("Valor es " , h) # dar el valor de hipotesos
print("Valor esperado " , x1["target"])  # verificar si era el mismo

sepal length (cm)    5.1
sepal width (cm)     3.5
petal length (cm)    1.4
petal width (cm)     0.2
target               0.0
Name: 0, dtype: float64
Valor es  0.0
Valor esperado  0.0


Parece que todo es correcto. Dados los valores a,b,c,d (nuestros coeficientes puestos arbitrariamente), el valor esperado de salida es igual al valor propuesto por nuestro agente. Es importante identificar la línea x1=iris.iloc\[0\]. Pandas tiene dos maneras de obtener las filas, por número de fila y por índice de fila (los índices pueden ser palabras completas). En este caso estamos accediendo a la primera fila basado en el número. A partir de ese punto x1 tiene una __Serie__, es decir, un tipo de objeto que representa una fila. Y para acceder a sus datos el método es como si fuera una lista o arreglo. Por eso la función agent puede hacer x\[0\], es decir, el primer dato de la serie. Por cierto print(x1) está imprimiendo lo que aparenta ser una tabla con el nombre del estímulo y el valor del mismo para la serie x1. Esto es solo una manera de visualizarlo propuesta por pandas - por eso fue creado, para hacer más fácil el acceso y visualización. Para accederlo lo puedes entender como un arreglo.
Para implementar la función de rendimiento:


In [ ]:
# definición de funcion de rendimiento
def performance(h,y):
  if( h != y):
    return 1
  return 0


p = 0 # acumulador para cantidad de resultados incorrectos
for index, x in iris.iterrows(): # iteracion por cada una de las filas
  h = agent(0,0,0,0,x) # obtención de respuesta
  y = x["target"] # acceso al valor real de x
  p += performance(h,y) # obtencion de rendimiento
print("Rendimiento" , p)

Rendimiento 100


Mientras la función de rendimiento solo evalúa si son iguales, el ciclo de acceso y evaluación de los datos nos permite aplicar la función del agente a lo largo de las 150 filas. Es importante analizar el iterador iterrows(). Éste entrega, por cada registro, una tupla de dos valores. El primero es el índice de la fila y el segundo es la serie de los datos. Con la variable x tenemos la serie necesaria para la función del agente que evalúa la ecuación que definimos. Finalmente, accedemos al valor de clasificación original de x, con el nombre de la columna – otra ventaja de las series de pandas, acceder con el nombre. Finalmente la función de performance nos entra un 1 cuando es incorrecto y 0 cuando hemos atinado. Rendimiento de 100, es increíble ¿no?

Hay una serie de errores en esta implementación que son relativamente comunes en IA:

1. El rendimiento entrega valores de error. Es decir, nos interesa saber el valor complementario para determinar cuántas clasificaciones correctas se hicieron.
2. La función del agente entrega valores flotantes, mientras que la performance evalúa valores enteros. Esto generará un problema cuando el agente empiece a tener valores en a,b,c,d entre 0 y 1.





Modificando para resolver los elementos anteriores:

In [ ]:
# definicion del agente, mapeo de sensores a clasificacion
def agent_int(a,b,c,d,x):
  return int(a*x[0] + b*x[1]  + c*x[2] + d*x[3])


p = 0 # acumulador para cantidad de resultados incorrectos
for index, x in iris.iterrows(): # iteracion por cada una de las filas
  h = agent_int(0.5,0.5,0.5,0.5,x) # obtención de respuesta
  y = x["target"] # acceso al valor real de x
  p += performance(h,y) # obtencion de rendimiento

p = (iris.shape[0] - p)/iris.shape[0]
print("Rendimiento" , p)

Rendimiento 0.0


Además de agregar el int() al agente, hemos cambiado la manera de medir el rendimiento (no la función). Ahora, al final del ciclo de análisis, se restan los errores al tamaño de la base de datos. Iris.shape es una lista con la cantidad de filas y de columnas (shape[0] y shape[1]). Al dividir entre el total de filas se cuenta con un indicador porcentual. En este caso, 33%. Es decir, nuestro modelo puede predecir correctamente el 33% de los registros. ¿Es bueno? En general no, aunque hay problemas que son difíciles y los mejores algoritmos pueden llegar a tener rendimientos de 10 o 20%. Para este en problema en particular es muy malo, puesto que hay soluciones con redes neuronales que pueden llegar a un 99.99% de rendimiento [13].


¿Y cómo podemos mejorar la inteligencia de nuestro agente? ¿Recuerdas los parámetros a,b,c,d? Justo son para poder variar la importancia del estímulo y poder dar nuevos valores. Vuelve a probar el ciclo anterior modificando los valores a,b,c,d:
```pythhon
h = agent_int(0.5,0.5,0.5,0.5,x) # obtencion de respuesta
```

¡Y verás un rendimiento todavía peor! Es decir, poco racional con respecto a lo esperado. El problema más allá del modelo, está en las matemáticas básicas.
Mira esta muestra obtenida de la base de datos:



In [ ]:
iris.sample(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
113,5.7,2.5,5.0,2.0,2
103,6.3,2.9,5.6,1.8,2
47,4.6,3.2,1.4,0.2,0
61,5.9,3.0,4.2,1.5,1
109,7.2,3.6,6.1,2.5,2
58,6.6,2.9,4.6,1.3,1
84,5.4,3.0,4.5,1.5,1
141,6.9,3.1,5.1,2.3,2
72,6.3,2.5,4.9,1.5,1
77,6.7,3.0,5.0,1.7,1


Si haces la suma manualmente, rápidamente podrás identificar que los valores de entrada al ser multiplicados por 0.5, rápidamente darán valores muy por encima de 2. Entonces la ecuación no está generando valores válidos. Antes era correcto en 1/3 de las ocasiones porque siempre generaba 0. ¿Cómo se puede resolver esto? Hay dos grandes maneras:

1. Escalando/Transformando el valor de salida: Es decir, en lugar de que puedan salir números como 100, tratar de ver cuál es el máximo y dividir la respuesta de agent_int entre ese valor.

2. Escalando/Transformando las entradas: Probablemente si todos los valores de los sensores se pueden llevar a un mismo formato, sea más fácil controlar la salida.

La opción (2) es la recurrente. Hay varios procesos estadísticos para poder hacer esto, nosotros usaremos la normalización max-min. En general, tomaremos el supuesto que en la base de datos existe el mayor y menor valor que las plantas pueden tener por cada sensor (que también puede ser investigado en internet). Restaremos cada valor al mínimo de cada columna para dividirlo entre la diferencia del mayor menos el menor. ¿Suena complejo? Pandas lo hace muy fácil, dado que es una operación muy normal en este campo:


In [ ]:
print("Base de datos ANTES DE CONVERSION \n ---------------------------------------\n")
print(iris) # imprime la base de datos ANTES
target = iris["target"] # guarda la columna de target por que la sig. operacion es sobre la base de datos

iris = (iris - iris.min())/(iris.max() - iris.min() ) # normaliza max min

iris["target"] = target # reasigna target

print("Base de datos DESPUES DE CONVERSION \n ---------------------------------------\n")
print(iris) # imprime

Base de datos ANTES DE CONVERSION 
 ---------------------------------------

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9          

Ahora tenemos más control sobre los valores de entrada. No necesariamente está es la mejor manera de trabajar con los datos. Justo por esta razón es que diferentes modelos se adaptan a unos u otros problemas.
Pasemos un poco a las matemáticas. Con estos 4 valores normalizados, el máximo que podemos obtener en cualquier suma dados a,b,c,d igual a 1, es 4. En general, el máximo valor es igual a la suma de a,b,c y d. Vamos a dividir la función del agente entre esta suma y luego escalar la salida a 0,1 o 2 para  ver qué pasa. Mira esta reimplementación de agent:

In [ ]:
# definicion del agente, mapeo de sensores a clasificacion
def agent_scaled(a,b,c,d,x):
  s = a+b+c+d+.0000001
  return int(  ((a*x[0] + b*x[1]  + c*x[2] + d*x[3])/s)*3 )


p = 0 # acumulador para cantidad de resultados incorrectos
for index, x in iris.iterrows(): # iteracion por cada una de las filas
  h = agent_scaled(0.5,0.5,0.5,0.5,x) # obtención de respuesta
  y = x["target"] # acceso al valor real de x
  p += performance(h,y) # obtencion de rendimiento

p = (iris.shape[0] - p)/iris.shape[0]
print("Rendimiento" , p)

Rendimiento 0.7733333333333333


Hemos mejorado el rendimiento trascendentalmente, aunque tal vez, no suficiente para competir mundialmente. Esto también significa que, de cada 10 plantas que entren a tu sistema en el invernadero, 7 tendrían un diagnóstico correcto con respecto a la cantidad de riego que necesitan. Puedes variar los valores de a,b,c,d para ver si logras un mejor rendimiento. Como dato, en pruebas logramos 96% y fue dependiente de solamente 1 de los parámetros.

## Fin del código

Te dejamos las actividades extras pero antes, sigue con la parte del contenido restante en la plataforma.

# Actividades Extras

__Actividad 1__

1. Separa un 30% de los datos.  Lo puedes hacer con pandas:
```
iris_test_ds = iris.sample(frac = 0.3)
iris_train_ds_index = iris.index.difference(iris_test_ds.index)
iris_train_ds = iris.loc[ iris_train_ds_index ]
```

2. Implementa un ciclo para ir evaluando cada uno de los 4 factores. El objetivo es probar con agent_scaled y  medir el performance solo en este subconjunto.
3. Una vez que tengas la mejor combinación, prueba con la base de datos completa. Nota: idealmente tendrías que probar en el restante 70% pero vamos a obviar este detalle por el momento.

Responde estas preguntas

1. Este pre-estudio, ¿ayuda para determinar los coeficientes?
2. ¿En qué casos consideras útil tener este proceso?



# AYUDA

El siguiente pedazo de código implementa el ciclo de la actividad. Recibe un Dataset. Trata primero de resolver la actividad de manera autónoma.


## El código presentado es una ayuda:

In [ ]:
def do_train(a,b,c,d, ds):
  r = 0
  p = 0
  for index, x in ds.iterrows():
    h = agent_scaled(a,b,c,d,x) # obtención de respuesta
    y = x["target"] # acceso al valor real de x
    p += performance(h,y)


  p = (ds.shape[0] - p)/ds.shape[0]
  return p

resultsA = []
resultsB = []
resultsC = []
resultsD = []
resultsE = []

iterations = 0
start = 0
end = 10
for a in range(start,end):
  for b in range(start,end):
    for c in range(start,end):
      for d in range(start,end):
        acc = do_train(a/10,b/10,c/10,d/10, iris)
        resultsA.append(a/10)
        resultsB.append(b/10)
        resultsC.append(c/10)
        resultsD.append(d/10)
        resultsE.append(acc)
        iterations += 1
  print( (iterations/100) , "% out of " , 100 , "%")

dct = {"a":resultsA,"b":resultsB,"c":resultsC,"d":resultsD,"acc":resultsE }
results = pd.DataFrame(dct)


10.0 % out of  100 %
20.0 % out of  100 %
30.0 % out of  100 %
40.0 % out of  100 %
50.0 % out of  100 %
60.0 % out of  100 %
70.0 % out of  100 %
80.0 % out of  100 %
90.0 % out of  100 %
100.0 % out of  100 %


In [ ]:
print(results)

results.describe()

        a    b    c    d       acc
0     0.0  0.0  0.0  0.0  0.333333
1     0.0  0.0  0.0  0.1  0.960000
2     0.0  0.0  0.0  0.2  0.960000
3     0.0  0.0  0.0  0.3  0.960000
4     0.0  0.0  0.0  0.4  0.960000
...   ...  ...  ...  ...       ...
9995  0.9  0.9  0.9  0.5  0.720000
9996  0.9  0.9  0.9  0.6  0.740000
9997  0.9  0.9  0.9  0.7  0.766667
9998  0.9  0.9  0.9  0.8  0.766667
9999  0.9  0.9  0.9  0.9  0.773333

[10000 rows x 5 columns]


,a,b,c,d,acc
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.450000,0.450000,0.450000,0.450000,0.726431
std,0.287242,0.287242,0.287242,0.287242,0.151583
min,0.000000,0.000000,0.000000,0.000000,0.173333
25%,0.200000,0.200000,0.200000,0.200000,0.646667
50%,0.450000,0.450000,0.450000,0.450000,0.766667
75%,0.700000,0.700000,0.700000,0.700000,0.840000
max,0.900000,0.900000,0.900000,0.900000,0.966667
